# SAS Innovate Workbench Workshop

## 1. Importing Packages

In [1]:
import os
import pandas as pd

# 2. Data preparation

In [2]:
# List all files in the directory
files = os.listdir()

# Filter files that end with .csv
csv_files = [f for f in files if f.endswith('.csv')]

print(csv_files)

['Bank_Score.csv', 'Bank_Train.csv']


In [3]:
# Load each CSV file as a DataFrame
Bank_Score = pd.read_csv(csv_files[0])
Bank_Train = pd.read_csv(csv_files[1])

In [4]:
# displaying dataframe
Bank_Train.head()

,AccountID,Status,Customer_Value,Demog_Age,Home_Flag,Demog_Homeval,demog_inc,demog_pr,Activity_Status,AvgSale3Yr,...,AvgSale3Yr_DP,LastProdAmt,CntPur3Yr,CntPurLife,CntPur3Yr_DP,CntPurLife_DP,CntTotPromo,MnthsLastPur,Cnt1Yr_DP,CustTenure
0,5200000001,1,A,NaN,0,57600,52106,24,High,5.71,...,5.25,10.0,7,22,4,6,20,5,9,92
1,5200000002,1,A,NaN,0,57587,52106,24,High,5.71,...,5.25,10.0,7,22,4,6,20,5,9,92
2,5200000003,1,A,NaN,0,44167,42422,0,High,12.80,...,13.00,12.0,5,16,3,8,27,16,11,91
3,5200000004,0,A,68.0,0,90587,59785,32,High,21.60,...,20.00,25.0,5,21,2,7,19,15,9,123
4,5200000005,0,A,NaN,0,100313,0,0,High,7.33,...,7.60,10.0,6,38,5,19,13,24,6,128


In [5]:
# displaying dataframe
Bank_Score.head()

,AccountID,Customer_Value,Demog_Age,Home_Flag,Demog_Homeval,demog_inc,demog_pr,AvgSale3Yr,AvgSaleLife,AvgSale3Yr_DP,LastProdAmt,CntPur3Yr,CntPurLife,CntPur3Yr_DP,CntPurLife_DP,CntTotPromo,MnthsLastPur,Cnt1Yr_DP,CustTenure,Activity_Status
0,5200200001,A,NaN,0,450010,0,9,20.0,20.00,20.00,20.0,0,1,0,1,10,20,3,39,High
1,5200200002,A,NaN,0,350025,0,14,23.0,12.78,25.00,24.0,3,9,1,3,13,18,6,65,High
2,5200200003,B,NaN,0,228806,0,12,17.5,13.67,17.50,20.0,1,2,1,1,11,18,5,27,Average
3,5200200004,A,NaN,0,240822,0,13,13.0,13.00,10.00,16.0,2,2,1,1,10,20,4,32,High
4,5200200005,A,NaN,1,189808,53669,25,11.0,11.00,11.67,10.0,5,5,3,3,15,15,6,33,High


Note that Status column is absent in the Score set.

We will have to use the Train set for training and testing, and the Score set can only be used as new unseen data.

In [6]:
# rename datasets to align with project needs
bank = Bank_Train
bank_unseen = Bank_Score

## 3. Exploratory Data Analysis (EDA)

In [7]:
# understanding datasets dimensions
print("Bank_Train data shape:", bank.shape)
print("Bank_Score data shape:", bank_unseen.shape)

Bank_Train data shape: (108600, 21)
Bank_Score data shape: (225068, 20)


Note Score set is bigger than Train.

*DISCUSSION:* Can consider adding synthetic data to Train set.

In [8]:
# understanding column format
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108600 entries, 0 to 108599
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AccountID        108600 non-null  int64  
 1   Status           108600 non-null  int64  
 2   Customer_Value   108600 non-null  object 
 3   Demog_Age        81143 non-null   float64
 4   Home_Flag        108600 non-null  int64  
 5   Demog_Homeval    108600 non-null  int64  
 6   demog_inc        108600 non-null  int64  
 7   demog_pr         108600 non-null  int64  
 8   Activity_Status  108600 non-null  object 
 9   AvgSale3Yr       108600 non-null  float64
 10  AvgSaleLife      108600 non-null  float64
 11  AvgSale3Yr_DP    85445 non-null   float64
 12  LastProdAmt      108600 non-null  float64
 13  CntPur3Yr        108600 non-null  int64  
 14  CntPurLife       108600 non-null  int64  
 15  CntPur3Yr_DP     108600 non-null  int64  
 16  CntPurLife_DP    108600 non-null  int6

In [9]:
# missing variable exploration
bank.isna().sum()

AccountID              0
Status                 0
Customer_Value         0
Demog_Age          27457
Home_Flag              0
Demog_Homeval          0
demog_inc              0
demog_pr               0
Activity_Status        0
AvgSale3Yr             0
AvgSaleLife            0
AvgSale3Yr_DP      23155
LastProdAmt            0
CntPur3Yr              0
CntPurLife             0
CntPur3Yr_DP           0
CntPurLife_DP          0
CntTotPromo            0
MnthsLastPur           0
Cnt1Yr_DP              0
CustTenure             0
dtype: int64

Only Demog_Age and AvgSale3Yr_DP have missing data, but missing 20-25% of data.

In [10]:
# finding duplicated data
bank.duplicated().sum()

0

In [11]:
# summary statistics
bank.describe().T

,count,mean,std,min,25%,50%,75%,max
AccountID,108600.0,5.200054e+09,31350.263954,5.200000e+09,5.200027e+09,5.200054e+09,5.200081e+09,5.200109e+09
Status,108600.0,2.067495e-01,0.404976,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
Demog_Age,81143.0,5.879187e+01,16.912909,0.000000e+00,4.600000e+01,6.000000e+01,7.300000e+01,8.800000e+01
Home_Flag,108600.0,5.489411e-01,0.497601,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Demog_Homeval,108600.0,1.132815e+05,99259.281461,0.000000e+00,5.300775e+04,7.808350e+04,1.352010e+05,6.000490e+05
demog_inc,108600.0,4.045955e+04,28388.996919,0.000000e+00,2.465375e+04,4.321300e+04,5.722300e+04,2.000070e+05
demog_pr,108600.0,2.963630e+01,12.112024,0.000000e+00,2.400000e+01,3.000000e+01,3.700000e+01,1.000000e+02
AvgSale3Yr,108600.0,1.619683e+01,18.744587,-1.000000e+00,1.000000e+01,1.500000e+01,2.000000e+01,3.713310e+03
AvgSaleLife,108600.0,1.347162e+01,9.767661,1.580000e+00,8.480000e+00,1.167000e+01,1.567000e+01,6.500000e+02
AvgSale3Yr_DP,85445.0,1.542408e+01,18.420151,3.200000e-01,1.000000e+01,1.400000e+01,2.000000e+01,3.713310e+03
